# Ranch Demo Notebook

This notebook has examples of how to make Ranch calls to run through the complete network building pipeline to develop standard roadway and transit network from scratch

In [5]:
import os
import pickle
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import json
import sys
import numpy as np

import ranch
from ranch import sharedstreets
from ranch import Roadway
from ranch import Transit
from ranch import Parameters
from ranch.utils import link_df_to_geojson, point_df_to_geojson
from ranch.logger import RanchLogger

In [6]:
%load_ext autoreload
%autoreload 2

# Remote I/O

User to update, project directory is not necessarily the ranch directory

In [7]:
# project directory
root_dir = os.path.join("D:/metcouncil_network_rebuild")

external_dir = os.path.join(root_dir, "data", "external")
interim_dir = os.path.join(root_dir, "data", "interim")

# software directory
ranch_dir = os.path.join("D:/github/Ranch")

# the folder where SharedStreets extractions live

shst_extract_dir = os.path.join(external_dir, "sharedstreets_extract")

# the folder where OSMNX extractions live

osm_extract_dir = os.path.join(external_dir, "osmnx_extract")

In [8]:
parameters = Parameters(ranch_base_dir = ranch_dir)

2022-03-07 12:22:16, INFO: Lasso base directory set as: D:/github/Ranch


In [9]:
parameters.standard_crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
for d in [root_dir, external_dir, interim_dir, shst_extract_dir, osm_extract_dir]:
    if os.path.isdir(d):
        print(f"{d} exists!")
        continue
    
    print(f"creating {d}")
    os.makedirs(d)

D:/metcouncil_network_rebuild exists!
D:/metcouncil_network_rebuild\data\external exists!
D:/metcouncil_network_rebuild\data\interim exists!
D:/metcouncil_network_rebuild\data\external\sharedstreets_extract exists!
D:/metcouncil_network_rebuild\data\external\osmnx_extract exists!


## Step 1 - SharedStreets Extraction

Extracts sharedstreets representation of OSM links.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from SharedStreets API.

In [11]:
# the polygon file for the area

input_polygon_file = os.path.join(
    external_dir,
    "county", 
    "cb_2018_us_county_500k_withinMETC.geojson"
)

In [10]:
# the SharedStreets extraction file is saved to the 'output_dir' argument

ranch.run_shst_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = shst_extract_dir
)

2022-03-06 17:01:31, INFO: Exporting boundry file D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\boundary.0.geojson
2022-03-06 17:01:31, INFO: extracting for polygon 0
2022-03-06 17:02:56, INFO: Exporting boundry file D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\boundary.1.geojson
2022-03-06 17:02:56, INFO: extracting for polygon 1
2022-03-06 17:03:38, INFO: Exporting boundry file D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\boundary.2.geojson
2022-03-06 17:03:38, INFO: extracting for polygon 2
2022-03-06 17:04:37, INFO: Exporting boundry file D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\boundary.3.geojson
2022-03-06 17:04:37, INFO: extracting for polygon 3
2022-03-06 17:05:50, INFO: Exporting boundry file D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\boundary.4.geojson
2022-03-06 17:05:50, INFO: extracting for polygon 4
2022-03-06 17:06:50, INFO: Exporting boundry file D:/metcouncil_n

## Step 2 - OSMNX Extraction

Extracts complete OSM attributes using OSMNX.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from OSMNX.

In [11]:
# the OSMNX extraction file is saved to the 'output_dir' argument

ranch.run_osmnx_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = os.path.join(external_dir, "osmnx_extract")
)

## Step 3 - Consolidate SharedStreets and OSMNX extractions

consolidates raw roadway data and create initial roadway networks

In [13]:
# build and returns roadway network object from extraction files

metc_roadway_network = Roadway.create_roadway_network_from_extracts(
    shst_extract_dir = os.path.join(shst_extract_dir),
    osm_extract_dir = osm_extract_dir,
    parameters = parameters,
)

2022-03-07 13:01:45, INFO: Reading sharedstreets data
2022-03-07 13:01:45, INFO: ----------start reading shst extraction data-------------
2022-03-07 13:01:45, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\extract.boundary.0.out.geojson
2022-03-07 13:02:29, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\extract.boundary.1.out.geojson
2022-03-07 13:02:36, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\extract.boundary.10.out.geojson
2022-03-07 13:02:39, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\extract.boundary.11.out.geojson
2022-03-07 13:02:44, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\sharedstreets_extract\extract.boundary.12.out.geojson
2022-03-07 13:02:46, INFO: reading shst extraction data : D:/metcouncil_network_reb

In [14]:
RanchLogger.info("Initial network has {} links".format(metc_roadway_network.links_df.shape[0]))
RanchLogger.info("Initial network has {} nodes".format(metc_roadway_network.nodes_df.shape[0]))
RanchLogger.info("Initial network has {} shapes".format(metc_roadway_network.shapes_df.shape[0]))

2022-03-07 14:20:54, INFO: Initial network has 1099287 links
2022-03-07 14:20:54, INFO: Initial network has 420408 nodes
2022-03-07 14:20:54, INFO: Initial network has 576818 shapes


### Optional: If the user wants to write out the roadway network in standard format, they can do the following, otherwise no need to write out. These files can also be written out after step 5.  After step 5, roadway network is trimmed to be within county boundary

In [ ]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step3_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step3_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link json")

link_prop = roadway_network.links_df.drop(
    ['geometry'], 
    axis = 1
).columns.tolist()

out = roadway_network.links_df[link_prop].to_json(orient = "records")

with open(os.path.join(interim_dir,"step3_links.json"), 'w') as f:
    f.write(out)
    
# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step3_links.geojson"), "w") as f:
    json.dump(link_geojson, f)

### Optional: it's more efficient if the user writes out the intermediate network object in pickle

In [15]:
working_network_filename = os.path.join(interim_dir,"step3_network.pickle")
pickle.dump(metc_roadway_network, open(working_network_filename, 'wb'))

## Step 4 - Third-Party Roadway Data Conflation

Conflates e.g. county network with roadway network using SharedStreets.
The input for this step is the 3rd-party geodatabases to conflate.
The output of this step is the SharedStreets conflation outputs in geojson.

#### Metcouncil's reviewed version of base network

In [46]:
# define the input 3rd-party file, either shapefile or geojson

metc_network_file = os.path.join(external_dir, "rachel_corrected_version", "BaseNet_NoMnPASS.shp")

In [47]:
metc_network_gdf = gpd.read_file(metc_network_file)

In [48]:
metc_network_gdf.crs

<Projected CRS: EPSG:26915>
Name: NAD83 / UTM zone 15N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 96°W and 90°W - onshore and offshore. Canada - Manitoba; Nunavut; Ontario. United States (USA) - Arkansas; Illinois; Iowa; Kansas; Louisiana; Michigan; Minnesota; Mississippi; Missouri; Nebraska; Oklahoma; Tennessee; Texas; Wisconsin.
- bounds: (-96.0, 25.61, -90.0, 84.0)
Coordinate Operation:
- name: UTM zone 15N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [49]:
# conflate just the drive links, exclude centroid connectors

metc_network_gdf = metc_network_gdf[
    (metc_network_gdf.drive == 1) & 
    (metc_network_gdf.A > 3100) & 
    (metc_network_gdf.B > 3100)
]

In [50]:
metc_network_gdf[metc_network_gdf.geoId == '4a683887505ce9fd1f7a2be95f3956ca']

,link_id,geoId,shape_id,A,B,distance,roadway,name,drive,walk,...,lanes_NT,m_lanes_AM,m_lanes_MD,m_lanes_PM,m_lanes_NT,access_AM,access_MD,access_PM,access_NT,geometry
42634,42635,4a683887505ce9fd1f7a2be95f3956ca,7b041916e6fd3744a45497c9a30ce7ca,23578,49818,0.254237,tertiary,East Travelers Trail,1,0,...,2,0,0,0,0,None,None,None,None,"LINESTRING (478906.009 4958547.454, 478892.066..."
104567,104568,4a683887505ce9fd1f7a2be95f3956ca,e433e4c6ede17230343b758534cfee29,49818,23578,0.254237,tertiary,East Travelers Trail,1,0,...,2,0,0,0,0,None,None,None,None,"LINESTRING (478551.595 4958343.009, 478580.945..."


In [51]:
metc_network_gdf.county.value_counts(dropna = False)

4     88223
5     37530
3     32847
1     30391
7     23361
22    14055
6     11861
2     11092
21    10794
17     9263
19     8680
18     7832
12     7632
15     7223
11     6467
13     6256
16     6118
14     5126
20     4998
10      204
Name: county, dtype: int64

In [66]:
county_polygon_file = os.path.join(
    external_dir,
    "county", 
    "cb_2018_us_county_500k_withinMETC.geojson"
)

county_polygon_gdf = gpd.read_file(county_polygon_file)

In [70]:
for i in range(len(county_polygon_gdf)):
    
    boundary_gdf = gpd.read_file(os.path.join(external_dir, "sharedstreets_extract/boundary."+str(i)+".geojson"))
    
    metc_network_gdf = metc_network_gdf.to_crs(CRS('epsg:4269'))
    
    sub_gdf = metc_network_gdf[metc_network_gdf.intersects(boundary_gdf.geometry.unary_union)].copy()
    
    metc_input_network_file = os.path.join(
        external_dir, 'rachel_corrected_version', "shst_match", "county_"+str(i)+"_in.geojson"
    )
    
    sub_gdf[['A', 'B', "geometry"]].to_file(
        os.path.join(metc_input_network_file), 
        driver = "GeoJSON"
    )

In [73]:
for i in range(len(county_polygon_gdf)):
    metc_input_network_file = os.path.join(
        external_dir, 'rachel_corrected_version', "shst_match", "county_"+str(i)+"_in.geojson"
    )

    ranch.run_shst_match(
        input_network_file = metc_input_network_file,
        input_unique_id = ['A', 'B'],
        output_dir = os.path.join(external_dir, 'rachel_corrected_version', "shst_match"),
        custom_match_option = '--tile-hierarchy=8 --search-radius=1 --snap-intersections --follow-line-direction'
    )

#### Anoka lanes

In [186]:
anoka_county_network_file = os.path.join(
    external_dir, 'CountyLanes_3-27-19', 'Anoka', 'AnokaCo_Segments_032519.shp'
)

In [187]:
anoka_county_network_df = gpd.read_file(anoka_county_network_file)

In [188]:
anoka_county_network_df.crs

<Projected CRS: PROJCS["NAD_1983_Lambert_Conformal_Conic",GEOGCS[" ...>
Name: NAD_1983_Lambert_Conformal_Conic
Axis Info [cartesian]:
- [east]: Easting (US survey foot)
- [north]: Northing (US survey foot)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [194]:
# check if there is unique ID
# Segment_ID is unique ID

print(anoka_county_network_df.shape)
print(anoka_county_network_df.Segment_ID.nunique())

(506, 51)
506


In [205]:
ranch.run_shst_match(
    input_network_file = anoka_county_network_file,
    input_unique_id = ['Segment_ID'],
    output_dir = os.path.join(external_dir, 'CountyLanes_3-27-19', 'Anoka', "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

here


#### Hennepin lanes

In [206]:
hennepin_county_network_file = os.path.join(
    external_dir, 'CountyLanes_3-27-19', 'Hennepin', 'Roadway_Width_032619.shp'
)

In [207]:
hennepin_county_network_df = gpd.read_file(hennepin_county_network_file)

In [209]:
hennepin_county_network_df.crs

<Projected CRS: PROJCS["HENNEPIN COUNTY",GEOGCS["GCS_User_Defined" ...>
Name: HENNEPIN COUNTY
Axis Info [cartesian]:
- [east]: Easting (US survey foot)
- [north]: Northing (US survey foot)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: D_User_Defined
- Ellipsoid: User_Defined_Spheroid
- Prime Meridian: Greenwich

In [212]:
# check if there is unique ID
# MAPIDINT is unique ID

print(hennepin_county_network_df.shape)
print(hennepin_county_network_df.MAPIDINT.nunique())

(3033, 10)
3033


In [214]:
ranch.run_shst_match(
    input_network_file = hennepin_county_network_file,
    input_unique_id = ['MAPIDINT'],
    output_dir = os.path.join(external_dir, 'CountyLanes_3-27-19', 'Hennepin', "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

#### Carver lanes

In [215]:
carver_county_network_file = os.path.join(
    external_dir, 'CountyLanes_3-27-19', 'Carver County', 'Carver_export.shp'
)

In [216]:
carver_county_network_df = gpd.read_file(carver_county_network_file)

In [221]:
# check if there is unique ID
# OBJECTID is unique ID

print(carver_county_network_df.shape)
print(carver_county_network_df.OBJECTID.nunique())

(95, 9)
95


In [256]:
# carver network is 3D, need to reduce dimension
from shapely.geometry import LineString

carver_county_network_df['geometry'] = carver_county_network_df['geometry'].apply(
    lambda g: LineString([xy[:2] for xy in list(g.coords)])
)

In [259]:
carver_county_network_df.to_file(
    os.path.join(
    external_dir, 'CountyLanes_3-27-19', 'Carver County', 'Carver_export.geojson',
    ),
    driver='GeoJSON'
)

In [260]:
ranch.run_shst_match(
    input_network_file = os.path.join(
    external_dir, 'CountyLanes_3-27-19', 'Carver County', 'Carver_export.geojson',
    ),
    input_unique_id = ['OBJECTID'],
    output_dir = os.path.join(external_dir, 'CountyLanes_3-27-19', 'Carver County', "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

#### Dakota lanes

In [222]:
dakota_county_network_file = os.path.join(
    external_dir, 'CountyLanes_3-27-19', 'Dakota', 'Dakota_export.shp'
)

In [223]:
dakota_county_network_df = gpd.read_file(dakota_county_network_file)

In [229]:
# check if there is unique ID
# OBJECTID is unique ID

print(dakota_county_network_df.shape)
print(dakota_county_network_df.OBJECTID.nunique())

(499, 15)
499


In [240]:
ranch.run_shst_match(
    input_network_file = dakota_county_network_file,
    input_unique_id = ['OBJECTID'],
    output_dir = os.path.join(external_dir, 'CountyLanes_3-27-19', 'Dakota', "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

#### Washington lanes

In [230]:
washington_county_network_file = os.path.join(
    external_dir, 'WCNumberofLanesApril2019', 'WCNumberofLanesApril2019.shp'
)

In [231]:
washington_county_network_df = gpd.read_file(washington_county_network_file)

In [234]:
# check if there is unique ID
# NO unique ID

print(washington_county_network_df.shape)
print(washington_county_network_df.Rd_Number.nunique())

(642, 12)
53


In [241]:
ranch.run_shst_match(
    input_network_file = washington_county_network_file,
    #input_unique_id = ['MAPIDINT'],
    output_dir = os.path.join(external_dir, 'WCNumberofLanesApril2019', "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

2022-03-20 18:07:51, INFO: Input network for shst match does not have unique IDs, generating unique IDs
2022-03-20 18:07:51, INFO: Generated 642 unique IDs for 642 links in the input network
2022-03-20 18:07:51, INFO: Exporting shst match input - ID-ed geometry file D:/metcouncil_network_rebuild\data\external\WCNumberofLanesApril2019\shst_match\WCNumberofLanesApril2019.geojson
2022-03-20 18:07:52, INFO: Exporting ID-ed network file D:/metcouncil_network_rebuild\data\external\WCNumberofLanesApril2019\shst_match\WCNumberofLanesApril2019.full.geojson


#### MDOT lanes

In [242]:
mdot_lanes_network_file = os.path.join(
    external_dir, 'shp_trans_lanes', 'Export_Output.shp'
)

In [243]:
mdot_lanes_network_df = gpd.read_file(mdot_lanes_network_file)

In [244]:
mdot_lanes_network_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 174621 entries, 0 to 174620
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   OBJECTID    174621 non-null  int64   
 1   FROM_DATE   174621 non-null  object  
 2   TO_DATE     39439 non-null   object  
 3   EVENT_ID    174621 non-null  object  
 4   ROUTE_ID    174621 non-null  object  
 5   FROM_MEASU  174621 non-null  float64 
 6   TO_MEASURE  174621 non-null  float64 
 7   EDITED_BY   81894 non-null   object  
 8   EDITED_DAT  81894 non-null   object  
 9   THROUGH_LA  174621 non-null  int64   
 10  ACCURACY_S  174621 non-null  int64   
 11  LOCERROR    174621 non-null  object  
 12  COUNTY_NAM  174621 non-null  object  
 13  COUNTY_COD  174621 non-null  object  
 14  SHAPE_Leng  174621 non-null  float64 
 15  geometry    174621 non-null  geometry
dtypes: float64(3), geometry(1), int64(3), object(9)
memory usage: 21.3+ MB


In [245]:
# check if there is unique ID
# OBJECTID is unique ID

print(mdot_lanes_network_df.shape)
print(mdot_lanes_network_df.OBJECTID.nunique())

(174621, 16)
174621


In [248]:
for i in range(len(county_polygon_gdf)):
    
    boundary_gdf = gpd.read_file(os.path.join(external_dir, "sharedstreets_extract/boundary."+str(i)+".geojson"))
    
    mdot_lanes_network_df = mdot_lanes_network_df.to_crs(CRS('epsg:4269'))
    
    sub_gdf = mdot_lanes_network_df[mdot_lanes_network_df.intersects(boundary_gdf.geometry.unary_union)].copy()
    
    mdot_lanes_input_network_file = os.path.join(
        external_dir, 'shp_trans_lanes', "shst_match", "county_"+str(i)+"_in.geojson"
    )
    
    sub_gdf[['OBJECTID', "geometry"]].to_file(
        os.path.join(mdot_lanes_input_network_file), 
        driver = "GeoJSON"
    )

In [ ]:
for i in range(len(county_polygon_gdf)):
    mdot_lanes_input_network_file = os.path.join(
        external_dir, 'shp_trans_lanes', "shst_match", "county_"+str(i)+"_in.geojson"
    )

    ranch.run_shst_match(
        input_network_file = mdot_lanes_input_network_file,
        input_unique_id = ['A', 'B'],
        output_dir = os.path.join(external_dir, 'shp_trans_lanes', "shst_match"),
        custom_match_option = '--tile-hierarchy=8 --search-radius=10 --snap-intersections'
    )

In [250]:
mdot_lanes_network_df[mdot_lanes_network_df.OBJECTID.isin([153616, 153823])]

,OBJECTID,FROM_DATE,TO_DATE,EVENT_ID,ROUTE_ID,FROM_MEASU,TO_MEASURE,EDITED_BY,EDITED_DAT,THROUGH_LA,ACCURACY_S,LOCERROR,COUNTY_NAM,COUNTY_COD,SHAPE_Leng,geometry
153615,153616,2014-01-28,None,145064,0400006595110011-D,4.749351,5.833757,AD\dewi1mic,2017-06-15,1,0,NO ERROR,Rice,66,1745.185063,"LINESTRING Z (-93.31266 44.30009 0.00000, -93...."
153822,153823,2014-01-28,None,5143,0400006595110011-I,4.749351,5.833757,AD\dewi1mic,2017-06-15,1,0,NO ERROR,Rice,66,1745.185063,"LINESTRING Z (-93.31266 44.30009 0.00000, -93...."


#### WDOT lanes

#### MDOT roadway type

#### WDOT roadway type

## Step 5 - Tidy Up Roadway

Trim network to be within county boundary. Identifies drive dead-ends, cul-de-secs, numbering links and nodes, etc.
The input for this step is the polygon file with subregion identifier, e.g. county.
This step labels each link and node with the county name.

In [148]:
# the roadway_network used for step 6 should be the entire region's network

step3_network_filename = os.path.join(
    interim_dir, 'step3_network.pickle'
)

metc_roadway_network = pickle.load(open(step3_network_filename, 'rb'))

In [149]:
# this is the input county polygon file with county names

county_boundary_file = os.path.join(
    external_dir,
    "county", 
    "cb_2018_us_county_500k_withinMETC.geojson"
)

# specify the column to look for county name

county_variable_name = 'NAME'

In [150]:
metc_roadway_network.parameters.model_centroid_node_id_reserve = 3100

In [151]:
metc_roadway_network.parameters

In [152]:
metc_roadway_network.parameters.model_centroid_node_id_reserve

3100

In [153]:
metc_roadway_network.tidy_roadway(
    county_boundary_file = county_boundary_file,
    county_variable_name = county_variable_name,
    create_node_link_id = True,
    county_based_numbering = False
)

2022-03-16 14:44:33, INFO: Starting Step 5 Tidy Roadway
2022-03-16 14:46:50, INFO: Joining network with county boundary file for ['Hennepin' 'Scott' 'Washington' 'Wright' 'Sherburne' 'Rice' 'Ramsey'
 'Carver' 'Isanti' 'Pierce' 'McLeod' 'Goodhue' 'Le Sueur' 'Sibley'
 'Chisago' 'St. Croix' 'Polk' 'Dakota' 'Anoka'] county
2022-03-16 14:49:11, INFO: Dropping links and nodes that are outside of ['Isanti' 'Dakota' 'Hennepin' 'Sibley' 'Carver' 'St. Croix' 'Washington'
 'Ramsey' 'Anoka' 'Le Sueur' 'Wright' 'Polk' 'Rice' 'Scott' 'Goodhue'
 'Chisago' 'McLeod' 'Sherburne' 'Pierce'] county
2022-03-16 14:49:17, INFO: Droppping circular links
2022-03-16 14:49:20, INFO: Flagging dead-end streets for drive network
2022-03-16 14:49:36, INFO: Making dead-end streets drive_access = 0
2022-03-16 14:50:47, INFO: Dropping alternative links between same AB nodes
2022-03-16 14:51:35, INFO: Numbering links and nodes using county based rules


In [154]:
RanchLogger.info("Initial network has {} links".format(metc_roadway_network.links_df.shape[0]))
RanchLogger.info("Initial network has {} nodes".format(metc_roadway_network.nodes_df.shape[0]))
RanchLogger.info("Initial network has {} shapes".format(metc_roadway_network.shapes_df.shape[0]))

2022-03-16 14:51:54, INFO: Initial network has 1056575 links
2022-03-16 14:51:54, INFO: Initial network has 412227 nodes
2022-03-16 14:51:54, INFO: Initial network has 553939 shapes


In [21]:
# save out the step 5 output as pickle file

working_network_filename = os.path.join(interim_dir,"step5_network.pickle")
pickle.dump(metc_roadway_network, open(working_network_filename, 'wb'))

In [155]:
# check if missing IDs

# if node missing shst id
print(metc_roadway_network.nodes_df.shst_node_id.isnull().sum())
print(metc_roadway_network.nodes_df.shst_node_id.nunique())
# if node missing osm id
print(metc_roadway_network.nodes_df.osm_node_id.isnull().sum())
# if node missing osm id
print(metc_roadway_network.nodes_df.osm_node_id.dtype)
print(metc_roadway_network.nodes_df[metc_roadway_network.nodes_df.osm_node_id == 0])
print(metc_roadway_network.nodes_df.osm_node_id.nunique())
# if node missing model node id
print(metc_roadway_network.nodes_df.model_node_id.nunique())

# if link missing 
print(metc_roadway_network.links_df.shstReferenceId.isnull().sum())
print(metc_roadway_network.links_df.shstReferenceId.nunique())
print(metc_roadway_network.links_df.model_link_id.nunique())

# if link missing node id
print(metc_roadway_network.links_df.fromIntersectionId.isnull().sum())
print(metc_roadway_network.links_df.toIntersectionId.isnull().sum())
print(metc_roadway_network.links_df.u.isnull().sum())
print(metc_roadway_network.links_df[metc_roadway_network.links_df.u == 0])
print(metc_roadway_network.links_df.v.isnull().sum())
print(metc_roadway_network.links_df[metc_roadway_network.links_df.v == 0])
print(len(set(metc_roadway_network.links_df.v.tolist() + metc_roadway_network.links_df.u.tolist())))

0
412227
0
int64
Empty GeoDataFrame
Columns: [osm_node_id, shst_node_id, geometry, county, drive_access, walk_access, bike_access, model_node_id]
Index: []
412227
412227
0
1056575
1056575
0
0
0
Empty GeoDataFrame
Columns: [shstReferenceId, id, shstGeometryId, fromIntersectionId, toIntersectionId, geometry, u, v, nodeIds, wayId, roadClass, oneWay, roundabout, link, oneway, lanes, ref, name, highway, service, width, maxspeed, access, junction, bridge, tunnel, landuse, area, key, forward, roadway, drive_access, walk_access, bike_access, county, length, model_link_id, A, B]
Index: []

[0 rows x 39 columns]
0
Empty GeoDataFrame
Columns: [shstReferenceId, id, shstGeometryId, fromIntersectionId, toIntersectionId, geometry, u, v, nodeIds, wayId, roadClass, oneWay, roundabout, link, oneway, lanes, ref, name, highway, service, width, maxspeed, access, junction, bridge, tunnel, landuse, area, key, forward, roadway, drive_access, walk_access, bike_access, county, length, model_link_id, A, B]
Index

In [36]:
# for visual checking

RanchLogger.info("write out node geojson")

node_prop = metc_roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    metc_roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step5_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)


# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = metc_roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    metc_roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step5_links.geojson"), "w") as f:
    json.dump(link_geojson, f)

2022-03-08 14:53:47, INFO: write out node geojson
2022-03-08 14:54:51, INFO: write out link geojson


## Step 6 - Build Transit Network from GTFS

Build standard transit network from GTFS

In [83]:
parameters.model_time_period = {
    "pk": {"start": 6, "end": 9},
    "op": {"start": 9, "end": 15},
}

parameters.model_time_enum_list = {
    "start_time": {
        "pk": "06:00:00",
        "op": "09:00:00",
    },
    "end_time": {
        "pk": "09:00:00",
        "op": "15:00:00",
    },
}

In [108]:
# the directory where all gtfs feeds are located
# e.g. gtfs_dir/sjrtd_2015_0127/routes.txt
# e.g. gtfs_dir/bart_2015_0127/routes.txt

gtfs_dir = os.path.join(external_dir, "gtfs")

In [109]:
# read gtfs into transit object

transit_network = Transit.load_all_gtfs_feeds(
    gtfs_dir = gtfs_dir,
    roadway_network= metc_roadway_network,
    parameters=parameters
)

2022-03-11 19:39:11, INFO: Excluding weekend-only services
2022-03-11 19:39:11, INFO: Read and get representative transit feed from: D:/metcouncil_network_rebuild\data\external\gtfs\MT2019
2022-03-11 19:39:26, INFO: Excluding weekend-only services
2022-03-11 19:39:26, INFO: Read and get representative transit feed from: D:/metcouncil_network_rebuild\data\external\gtfs\MV2019
2022-03-11 19:39:29, INFO: Lasso base directory set as: D:/github/Ranch


In [110]:
# main activities - routing buses, creating rails

transit_network.build_standard_transit_network(
    num_most_pattern = 2,
    multithread_shst_match = True,
    multithread_shortest_path = False,
)

2022-03-11 19:39:29, INFO: Getting representative trip for each route by time of day and direction...
     route_id                service_id                            trip_id  \
5689   515-95  AUG19-MVS-BUS-Weekday-01  15289929-AUG19-MVS-BUS-Weekday-01   
5692   515-95  AUG19-MVS-BUS-Weekday-01  15289930-AUG19-MVS-BUS-Weekday-01   
5698   515-95  AUG19-MVS-BUS-Weekday-01  15289937-AUG19-MVS-BUS-Weekday-01   
5706   515-95  AUG19-MVS-BUS-Weekday-01  15289949-AUG19-MVS-BUS-Weekday-01   
5742   515-95  AUG19-MVS-BUS-Weekday-01  15289970-AUG19-MVS-BUS-Weekday-01   
5748   515-95  AUG19-MVS-BUS-Weekday-01  15289974-AUG19-MVS-BUS-Weekday-01   
5754   515-95  AUG19-MVS-BUS-Weekday-01  15289978-AUG19-MVS-BUS-Weekday-01   
5757   515-95  AUG19-MVS-BUS-Weekday-01  15289980-AUG19-MVS-BUS-Weekday-01   

                                         trip_headsign  direction_id block_id  \
5689            Eastbound 515E  66th St / Vets Med Ctr             0     1930   
5692   Eastbound 515C  66th St / 

2022-03-11 20:11:39, INFO: 	Routing agency MT2019, trip 15249345-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:12:01, INFO: 	Routing agency MT2019, trip 15244684-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:12:31, INFO: 	Routing agency MT2019, trip 15289512-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:12:58, INFO: 	Routing agency MT2019, trip 15244113-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:13:29, INFO: 	Routing agency MT2019, trip 15285257-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:13:57, INFO: 	Routing agency MT2019, trip 15245153-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:14:38, INFO: 	Routing agency MT2019, trip 15245102-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:15:08, INFO: 	Routing agency MT2019, trip 15289618-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:15:36, INFO: 	Routing agency MT2019, trip 15245290-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:16:00, INFO: 	Routing agency MT2019, trip 15248913-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:16:30, INFO: 	Routing agency MT2019, trip 15250299-AUG19-MVS-BUS-Weekday-01
2022-03-11

2022-03-11 20:54:17, INFO: 	Routing agency MT2019, trip 15244690-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:54:39, INFO: 	Routing agency MT2019, trip 15247261-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:55:03, INFO: 	Routing agency MT2019, trip 15247527-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:55:30, INFO: 	Routing agency MT2019, trip 15245920-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:56:01, INFO: 	Routing agency MT2019, trip 15246238-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:56:26, INFO: 	Routing agency MT2019, trip 15245330-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:56:47, INFO: 	Routing agency MT2019, trip 15249691-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:57:11, INFO: 	Routing agency MT2019, trip 15246209-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:57:46, INFO: 	Routing agency MT2019, trip 15247243-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:58:10, INFO: 	Routing agency MT2019, trip 15245908-AUG19-MVS-BUS-Weekday-01
2022-03-11 20:58:40, INFO: 	Routing agency MT2019, trip 15273501-AUG19-MVS-SUB-Weekday-01
2022-03-11

2022-03-11 21:30:36, INFO: 	Routing agency MT2019, trip 15415168-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:31:07, INFO: 	Routing agency MT2019, trip 15248566-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:31:37, INFO: 	Routing agency MT2019, trip 15249923-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:31:52, INFO: 	Routing agency MT2019, trip 15249912-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:32:05, INFO: 	Routing agency MT2019, trip 15244470-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:32:33, INFO: 	Routing agency MT2019, trip 15701426-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:33:04, INFO: 	Routing agency MT2019, trip 15249797-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:33:19, INFO: 	Routing agency MT2019, trip 15273471-AUG19-MVS-SUB-Weekday-01
2022-03-11 21:33:48, INFO: 	Routing agency MT2019, trip 15247492-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:34:22, INFO: 	Routing agency MT2019, trip 15251573-AUG19-MVS-BUS-Weekday-01
2022-03-11 21:34:32, INFO: 	Routing agency MT2019, trip 15288887-AUG19-MVS-BUS-Weekday-01
2022-03-11

2022-03-11 22:05:24, INFO: 	Routing agency MT2019, trip 15249332-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:05:41, INFO: 	Routing agency MT2019, trip 15249708-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:06:04, INFO: 	Routing agency MT2019, trip 15289949-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:06:18, INFO: 	Routing agency MT2019, trip 15246245-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:06:31, INFO: 	Routing agency MT2019, trip 15273491-AUG19-MVS-SUB-Weekday-01
2022-03-11 22:06:58, INFO: 	Routing agency MT2019, trip 15289978-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:07:12, INFO: 	Routing agency MT2019, trip 15285118-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:07:30, INFO: 	Routing agency MT2019, trip 15251433-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:07:44, INFO: 	Routing agency MT2019, trip 15701441-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:08:12, INFO: 	Routing agency MT2019, trip 15247501-AUG19-MVS-BUS-Weekday-01
2022-03-11 22:08:24, INFO: 	Routing agency MT2019, trip 15701431-AUG19-MVS-BUS-Weekday-01
2022-03-11

...

2022-03-12 05:33:24, INFO: 	Routing agency ['MV2019'], trip ['053-RL-436.0S-0748-20190914-Weekday-03'], between stops
2022-03-12 05:33:38, INFO: 	Routing agency ['MV2019'], trip ['078-MA-440.0N-0831-20190914-Weekday-03'], between stops
2022-03-12 05:33:48, INFO: 	Routing agency ['MV2019'], trip ['062-S1-421.0W-0803-20190914-Weekday-03'], between stops
2022-03-12 05:34:01, INFO: 	Routing agency ['MV2019'], trip ['005-S1-442.1E-0817-20190914-Weekday-03'], between stops
2022-03-12 05:34:24, INFO: 	Routing agency ['MV2019'], trip ['055-RD-446.2S-0634-20190914-Weekday-03'], between stops
2022-03-12 05:34:43, INFO: 	Routing agency ['MV2019'], trip ['029-RL-444.2N-0804-20190914-Weekday-03'], between stops
2022-03-12 05:35:07, INFO: 	Routing agency ['MV2019'], trip ['024-RL-440.2N-0729-20190914-Weekday-03'], between stops
2022-03-12 05:35:18, INFO: 	Routing agency ['MV2019'], trip ['107-MA-445.5W-0751-20190914-Weekday-03'], between stops
2022-03-12 05:35:57, INFO: 	Routing agency ['MV2019'], t

2022-03-13 22:31:28, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_190006.matched.geojson
2022-03-13 22:31:28, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_20006.matched.geojson
2022-03-13 22:31:28, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_20008.matched.geojson
2022-03-13 22:31:28, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_20009.matched.geojson
2022-03-13 22:31:28, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_20010.matched.geojson
2022-03-13 22:31:28, INFO: reading shst extraction data : D:/metcounc

2022-03-13 22:31:30, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_7220004.matched.geojson
2022-03-13 22:31:30, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_7220005.matched.geojson
2022-03-13 22:31:30, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_7240001.matched.geojson
2022-03-13 22:31:30, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_7410002.matched.geojson
2022-03-13 22:31:30, INFO: reading shst extraction data : D:/metcouncil_network_rebuild\data\external\gtfs\gtfs_shape_for_shst_match\match.lines_from_shapes_MT2019_750002.matched.geojson
2022-03-13 22:31:30, INFO: reading shst extraction data : D:/

C:\Users\wangs1\AppData\Local\Continuum\anaconda3\envs\ranch\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2022-03-13 22:37:48, INFO: Creating frequency reference


In [111]:
# check if shortest path transit builder failed any transit shapes

transit_network.shortest_path_failed_shape_list

[]

In [112]:
# writes out standard transit network and rail links/nodes if present

transit_network.write_standard_transit(
    path = interim_dir
)

In [113]:
# write out bus route for review
bus_trip_link_df = gpd.GeoDataFrame(
    transit_network.bus_trip_link_df, 
    crs = metc_roadway_network.links_df.crs)

bus_trip_link_df.drop('wayId', axis = 1).to_file(
    os.path.join(interim_dir, 'bus_routing.geojson'),
    driver = 'GeoJSON'
)

## Step 7 - Build Centroid Connectors

builds centroid connectors from TAZ

### TAZ files

In [115]:
metc_taz_polygon_file = os.path.join(
    external_dir,
    'taz',
    'TAZ.shp'
)

metc_taz_centroid_file = os.path.join(
    external_dir,
    'taz',
    'TAZ_nodes.shp'
)

In [116]:
metc_taz_polygon_gdf = gpd.read_file(metc_taz_polygon_file)

metc_taz_centroid_gdf = gpd.read_file(metc_taz_centroid_file)

In [130]:
metc_taz_centroid_gdf['TAZ'] = metc_taz_centroid_gdf['N']
metc_taz_centroid_gdf['model_node_id'] = metc_taz_centroid_gdf['N']

In [132]:
metc_taz_centroid_gdf.to_file(
    os.path.join(
        external_dir,
        'taz',
        'TAZ_nodes.geojson'
    ),
    driver = 'GeoJSON'
)

In [128]:
metc_taz_polygon_gdf.head(2)

,OBJECTID,TAZ,POP2010,POPINHH10,PPH10,POPINGQ10,HH2010,EMP2010,NRET2010,RETAIL2010,...,EMP2040,NRET2040,RETAIL2040,TCFLAG,ESOURCE10,ESOURCE14,ESOURCE18,Shape_STAr,Shape_STLe,geometry
0,3031,1,86.0,86.0,2.87,0.0,30.0,1.0,1.0,0.0,...,6.0,4.0,2.0,1.0,LED2006,QCEW2014,LED2014,1.411847e+07,15327.348373,"POLYGON ((460555.993 5028987.573, 460851.077 5..."
1,3032,10,225.0,225.0,3.46,0.0,65.0,10.0,10.0,0.0,...,8.0,8.0,0.0,1.0,QCEW2010,QCEW2014,QCEW2017,2.539695e+06,6381.066223,"POLYGON ((479228.108 5025733.177, 479207.290 5..."


In [121]:
print(metc_taz_polygon_gdf.TAZ.nunique())
print(metc_taz_polygon_gdf.TAZ.min())
print(metc_taz_polygon_gdf.TAZ.max())

3030
1
3030


In [133]:
metc_taz_centroid_gdf.head(2)

,OBJECTID,N,X,Y,geometry,TAZ,model_node_id
0,1,1,462454.755934,5.027410e+06,POINT (462454.756 5027410.345),1,1
1,2,2,468423.045357,5.028244e+06,POINT (468423.045 5028244.252),2,2


### Build centroid connectors

In [156]:
metc_roadway_network.nodes_df.model_node_id.min()

3101

In [166]:
metc_roadway_network.build_centroid_connectors(
    build_taz_drive = True,
    build_taz_active_modes = True,
    input_taz_polygon_file = metc_taz_polygon_file,
    input_taz_node_file= os.path.join(
        external_dir,
        'taz',
        'TAZ_nodes.geojson'
    ),
    taz_unique_id='TAZ'
)

2022-03-17 10:22:29, INFO: input file D:/metcouncil_network_rebuild\data\external\taz\TAZ.shp has crs : epsg:26915
2022-03-17 10:22:30, INFO: input file D:/metcouncil_network_rebuild\data\external\taz\TAZ_nodes.geojson has crs : epsg:26915


d:\github\ranch\ranch\roadway.py:1363: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  num_connectors_per_centroid,
d:\github\ranch\ranch\roadway.py:1363: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  num_connectors_per_centroid,
d:\github\ranch\ranch\roadway.py:1363: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  num_connectors_per_centroid,


In [167]:
metc_roadway_network.taz_cc_link_gdf.to_file(
    os.path.join(interim_dir, 'temp_centroid_connectors.geojson'),
    driver = 'GeoJSON'
)

In [168]:
# write out to pickle

metc_roadway_network.taz_node_gdf.to_pickle(
    os.path.join(interim_dir, 'centroid_node.pickle'),
)
metc_roadway_network.taz_cc_link_gdf.to_pickle(
    os.path.join(interim_dir, 'cc_link.pickle'),
)
metc_roadway_network.taz_cc_shape_gdf.to_pickle(
    os.path.join(interim_dir, 'cc_shape.pickle'),
)

In [262]:
metc_roadway_network.taz_cc_link_gdf

,A,B,drive_access,walk_access,bike_access,shstGeometryId,id,u,v,fromIntersectionId,toIntersectionId,county,roadway,geometry
0,102455.0,1460.0,1,0,0,be68d5105155224803a916f1b7673a60,be68d5105155224803a916f1b7673a60,9.800498e+08,NaN,e3809600ae9687b10115883a13a3bf4c,NaN,NaN,taz,"LINESTRING (-93.28633 44.93045, -93.28632 44.9..."
1,128772.0,1460.0,1,0,0,bd96aa983487595e203ce16ab2a453a5,bd96aa983487595e203ce16ab2a453a5,8.144914e+08,NaN,8b73508201be5b4e066bf0d3fcac1048,NaN,NaN,taz,"LINESTRING (-93.28504 44.93045, -93.28632 44.9..."
2,173207.0,1460.0,1,0,0,a068b76ce96e7d6dc4b6a5856cda2d3a,a068b76ce96e7d6dc4b6a5856cda2d3a,9.800498e+08,NaN,d02eb23520161ef8cb557f486f8380b9,NaN,Hennepin,taz,"LINESTRING (-93.28632 44.92864, -93.28632 44.9..."
3,48855.0,1460.0,1,0,0,65f5d3a0b2dad2527c7edb91ca8c2a7e,65f5d3a0b2dad2527c7edb91ca8c2a7e,9.800498e+08,NaN,bc537bb413b239b928fb22ddbc9ed524,NaN,Hennepin,taz,"LINESTRING (-93.28632 44.93227, -93.28632 44.9..."
4,237105.0,1456.0,1,0,0,a1cbb219075cfc246b24c986b1e5ef48,a1cbb219075cfc246b24c986b1e5ef48,1.093356e+09,NaN,4119faffac788aa9b7ac34b6a607fae2,NaN,Hennepin,taz,"LINESTRING (-93.28117 44.92323, -93.28118 44.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47637,159.0,8026.0,0,1,1,a16bb0e4d23285b7e96814fa2c432681,a16bb0e4d23285b7e96814fa2c432681,NaN,186246302.0,NaN,067555186f6c616b4ca9848f3c9ba712,Anoka,taz,"LINESTRING (-93.07758 45.24171, -93.11184 45.2..."
47638,3017.0,322860.0,0,1,1,55e8eda676b507e1d010fbea7f162daf,55e8eda676b507e1d010fbea7f162daf,NaN,232012200.0,NaN,1f48d8eab7ccb91e8bc57a00a6ea5f62,Polk,taz,"LINESTRING (-92.34754 45.26688, -92.33156 45.2..."
47639,3017.0,317675.0,0,1,1,5b1892bfdaa7029e127795fa53b936a9,5b1892bfdaa7029e127795fa53b936a9,NaN,231970669.0,NaN,4e4ec8af820728314fee78b279f1fdea,Polk,taz,"LINESTRING (-92.31701 45.26949, -92.33156 45.2..."
47640,3017.0,305800.0,0,1,1,484279742712c53b3a856938bd8f320b,484279742712c53b3a856938bd8f320b,NaN,232046466.0,NaN,4a53b945986fb1eb0e16ca16c9d06cc5,Polk,taz,"LINESTRING (-92.29614 45.25259, -92.33156 45.2..."


In [261]:
metc_roadway_network.taz_node_gdf

,OBJECTID,N,X,Y,taz_id,model_node_id,geometry
0,1,1,462454.755934,5.027410e+06,1,1,POINT (-93.47971 45.39920)
1,2,2,468423.045357,5.028244e+06,2,2,POINT (-93.40351 45.40700)
2,3,3,466398.768445,5.026811e+06,3,3,POINT (-93.42928 45.39401)
3,4,4,469460.320755,5.026485e+06,4,4,POINT (-93.39015 45.39122)
4,5,5,470697.732820,5.026840e+06,5,5,POINT (-93.37436 45.39447)
...,...,...,...,...,...,...,...
3025,3026,3026,549229.165037,5.015010e+06,3026,3026,POINT (-92.37225 45.28687)
3026,3027,3027,550309.800325,5.015741e+06,3027,3027,POINT (-92.35839 45.29337)
3027,3028,3028,550659.781743,5.016609e+06,3028,3028,POINT (-92.35384 45.30115)
3028,3029,3029,537252.038875,5.020769e+06,3029,3029,POINT (-92.52453 45.33944)


## Step 8 - Write out Standard Format

In [170]:
# this is the input county polygon file with county names

county_boundary_file = os.path.join(
    external_dir,
    "county", 
    "cb_2018_us_county_500k_withinMETC.geojson"
)

# specify the column to look for county name

county_variable_name = 'NAME'

metc_roadway_network.standard_format(
    county_boundary_file = county_boundary_file,
    county_variable_name = county_variable_name
)

2022-03-18 10:49:39, INFO: Starting Step 8 creating roadway standard format


In [173]:
metc_roadway_network.links_df['rail_only'] = 0
metc_roadway_network.nodes_df['rail_only'] = 0

In [185]:
# save out the step 5 output as pickle file

working_network_filename = os.path.join(interim_dir,"step8_network.pickle")
pickle.dump(metc_roadway_network, open(working_network_filename, 'wb'))

In [174]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    metc_roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step8_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = metc_roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    metc_roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step8_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link json")

link_prop = metc_roadway_network.links_df.drop(
    ['geometry'], 
    axis = 1
).columns.tolist()

out = metc_roadway_network.links_df[link_prop].to_json(orient = "records")

with open(os.path.join(interim_dir,"step8_links.json"), 'w') as f:
    f.write(out)
    
# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = metc_roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    metc_roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step8_links.geojson"), "w") as f:
    json.dump(link_geojson, f)

2022-03-18 10:52:38, INFO: write out shape geojson
2022-03-18 10:55:02, INFO: write out node geojson
2022-03-18 10:56:30, INFO: write out link json
2022-03-18 10:57:07, INFO: write out link geojson


In [180]:
print(metc_roadway_network.links_df.shape)
print(metc_roadway_network.links_df.model_link_id.nunique())
print(metc_roadway_network.links_df.shstReferenceId.nunique())
print(sum(metc_roadway_network.links_df.shstGeometryId.notnull()))
print(metc_roadway_network.links_df.shstGeometryId.nunique())

(1056575, 41)
1056575
1056575
1056575
553939


In [181]:
print(metc_roadway_network.shapes_df.shape)
print(metc_roadway_network.shapes_df.id.nunique())

(553939, 9)
553939


In [182]:
print(metc_roadway_network.nodes_df.shape)
print(metc_roadway_network.nodes_df.model_node_id.nunique())
print(len(set(metc_roadway_network.links_df.A.tolist() + metc_roadway_network.links_df.B.tolist())))

(412227, 9)
412227
412227


In [183]:
metc_roadway_network.links_df.rail_only.value_counts(dropna = False)

0    1056575
Name: rail_only, dtype: int64

In [184]:
metc_roadway_network.nodes_df.rail_only.value_counts(dropna = False)

0    412227
Name: rail_only, dtype: int64